# Task 1: Advanced Objective Function and Use Case

## 1. 


Logistic Regression models the probability that a binary response variable 
\( y \in \{0,1\} \)
equals 1 given a vector of predictors (features) 
\( \mathbf{x} \in \mathbb{R}^d \).
We write this as:

$$
P(y=1 \mid \mathbf{x}; \boldsymbol{\beta}) 
= \sigma(\boldsymbol{\beta}^\top \mathbf{x}),
$$

where 
\(\sigma(z) = \frac{1}{1 + e^{-z}}\)
is the logistic (sigmoid) function and 
\(\boldsymbol{\beta}\)
is the parameter vector to be learned.

Given a dataset 
\(\{(\mathbf{x}_i, y_i)\}_{i=1}^N\),
the likelihood function under the logistic model is:

$$
L(\boldsymbol{\beta}) 
= \prod_{i=1}^N 
\bigl[\sigma(\boldsymbol{\beta}^\top \mathbf{x}_i)\bigr]^{y_i} 
\bigl[1 - \sigma(\boldsymbol{\beta}^\top \mathbf{x}_i)\bigr]^{(1 - y_i)}.
$$

Maximizing the log-likelihood rather than the likelihood itself gives:

$$
\ell(\boldsymbol{\beta}) 
= \log L(\boldsymbol{\beta}) 
= \sum_{i=1}^N \Bigl[ 
y_i \log \sigma(\boldsymbol{\beta}^\top \mathbf{x}_i) 
+ (1 - y_i)\log \bigl(1 - \sigma(\boldsymbol{\beta}^\top \mathbf{x}_i)\bigr)
\Bigr].
$$

Hence, the MLE objective is:

$$
\hat{\boldsymbol{\beta}}_{\text{MLE}} 
= \arg\max_{\boldsymbol{\beta}} 
\ell(\boldsymbol{\beta}).
$$

In practice, we typically minimize the **negative** log-likelihood, which is the cross-entropy loss:

$$
J(\boldsymbol{\beta}) 
= -\ell(\boldsymbol{\beta}) 
= -\sum_{i=1}^N \Bigl[
y_i \log \sigma(\boldsymbol{\beta}^\top \mathbf{x}_i) 
+ (1 - y_i)\log \bigl(1 - \sigma(\boldsymbol{\beta}^\top \mathbf{x}_i)\bigr)
\Bigr].
$$


In MAP estimation, we incorporate a prior on the parameters \( \boldsymbol{\beta} \). By Bayes’ rule, the posterior of \( \boldsymbol{\beta} \) given data \(D\) is:

$$
p(\boldsymbol{\beta} \mid D)
\propto 
p(D \mid \boldsymbol{\beta}) \, p(\boldsymbol{\beta}).
$$

Thus, the MAP estimate is:

$$
\hat{\boldsymbol{\beta}}_{\text{MAP}}
= \arg\max_{\boldsymbol{\beta}} 
\Bigl[\log p(D \mid \boldsymbol{\beta}) + \log p(\boldsymbol{\beta})\Bigr].
$$

For Logistic Regression with a Gaussian prior 
\(\mathcal{N}(\mathbf{0}, \sigma^2 \mathbf{I})\),
the posterior leads to:

$$
\hat{\boldsymbol{\beta}}_{\text{MAP}} 
= \arg\max_{\boldsymbol{\beta}} 
\left[
\ell(\boldsymbol{\beta})
- \frac{1}{2\sigma^2} \|\boldsymbol{\beta}\|^2
\right],
$$

which, in minimization form, is the negative log-likelihood plus an \(\ell_2\) penalty:

$$
J_{\text{MAP}}(\boldsymbol{\beta}) 
= 
-\sum_{i=1}^N \Bigl[
y_i \log \sigma(\boldsymbol{\beta}^\top \mathbf{x}_i)
+ (1 - y_i)\log \bigl(1 - \sigma(\boldsymbol{\beta}^\top \mathbf{x}_i)\bigr)
\Bigr] 
+ \lambda \|\boldsymbol{\beta}\|^2,
$$

where 
\(\lambda = \frac{1}{2\sigma^2}\).

**Key Difference from MLE:** MAP introduces a prior term (\(\log p(\boldsymbol{\beta})\)), often corresponding to a regularization term in practice, while MLE does not.

---

## 2.

### Fraud Detection Example

Suppose we want to predict whether a credit card transaction is **fraudulent** (\(y=1\)) or **legitimate** (\(y=0\)). Each transaction \(i\) is characterized by a feature vector \(\mathbf{x}_i\) (e.g., transaction amount, time of day, location, customer’s history) and a binary label \(y_i \in \{0,1\}\).

**Why Logistic Regression?**  
- Logistic Regression provides \( P(y=1 \mid \mathbf{x}) \), useful for ranking or thresholding on fraud probabilities.  
- The coefficients can be inspected to see which features influence the prediction most.  
- Training is typically fast, even for large datasets, and inference is straightforward.

Both Logistic Regression and Linear SVM produce linear decision boundaries in feature space. Key distinctions:
- Logistic Regression uses the log-likelihood (cross-entropy) loss, while Linear SVM uses hinge loss to maximize margin.  
- Logistic Regression outputs a calibrated probability, whereas SVM does not inherently provide one.

---

## 3. 

- **Correspondence to the Model:**  
  Each transaction \(i\) corresponds to \(\mathbf{x}_i\) (features) and a label \(y_i\) (fraud vs. not fraud). Logistic Regression uses \(\sigma(\boldsymbol{\beta}^\top \mathbf{x}_i)\) to model the probability of \(y_i=1\).

- **Assumptions:**  
  1. We assume each \((\mathbf{x}_i, y_i)\) pair is independently and identically distributed.  
  2. We assume the log-odds is linear in \(\mathbf{x}\).  
  3. If the data were perfectly separable, the parameters could become unbounded.

---

## References

- Bishop, C. M. (2006). *Pattern Recognition and Machine Learning*. Springer.  
- Murphy, K. P. (2012). *Machine Learning: A Probabilistic Perspective*. MIT Press.  
- Friedman, J., Hastie, T., \& Tibshirani, R. (2001). *The Elements of Statistical Learning*. Springer.  
- Ng, A. Y. (2004). *Feature selection, L1 vs. L2 regularization, and rotational invariance*. In *ICML*.


# Task 2: Dataset and Advanced EDA


----

## 1. Dataset Selection

Dataset: Wine Quality (Red)
Link: https://archive.ics.uci.edu/dataset/186/wine+quality

Description:

The dataset has 1599 observations (wines).
There are 11 numeric features (e.g., fixed acidity, volatile acidity, residual sugar, etc.) describing various chemical properties.
A 12th column represents the wine quality (an integer score).




## 2. Exploratory Data Analysis

In [68]:
import os
import pdb
import time
import torch
import random
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn as nn
import matplotlib.pyplot as plt
from os import listdir
from os.path import join, exists
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from scipy.signal import savgol_filter
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

Shape: Should be (1599, 12).
Columns: 11 features + “quality.”
All features and the quality score are numeric. We confirm this as follows:

In [ ]:
###########################################################Basic Data Analysis###########################################################


# Reading the wine dataset 
url = "./winequality_red.csv"
data = pd.read_csv(url, sep=';')

# Checking shape, columns, initial info
print("Shape:", data.shape)
print("Columns:", data.columns)
data.info()

# Quick look at a few rows
data.head()

Checking  min/max/mean/std for each feature.
Verifying whether any features look suspicious (e.g., extremely large standard deviations).
The Wine Quality dataset typically has no missing values. We verify this as follows:

In [ ]:
############################################################Descriptive Statistics and Missing Values#####################################

data.describe()
data.isnull().sum()

Next, we check pairwise correlations among features. Features with very high correlation can inflate each other’s importance and cause instability in certain models.

In [ ]:
corr_matrix = data.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title("Correlation Matrix for Wine Features")
plt.show()

To quantitatively diagnose multicollinearity, we can compute the VIF for each numeric feature. A VIF above ~5 or 10 often indicates problematic multicollinearity.The newly computed VIFs are more realistic. We confirm  this as follows:

In [ ]:
X = data.drop('quality', axis=1)
X = sm.add_constant(X)
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [
    variance_inflation_factor(X.values, i) 
    for i in range(X.shape[1])
]
vif_data = vif_data[vif_data["feature"] != "const"]
vif_data


## 3. Data Visualization


Next, we show a pair plot describing kernel density estimates (KDE) on the diagonal. This can  help visualize relationships between features and their distributions. The diagonal shows KDE plots of each feature’s distribution. Off-diagonal scatter plots show pairwise relationships.

In [ ]:
sns.pairplot(data, 
             vars=['fixed acidity','volatile acidity','citric acid','residual sugar'],
             diag_kind='kde',
             plot_kws={'alpha': 0.3})
plt.suptitle("Pair Plot with KDE for Selected Features", y=1.02)
plt.show()


A clustermap (hierarchical clustering) can group features by similarity, which may reveal patterns in correlated features.  method='ward' uses Ward’s hierarchical clustering on the correlation matrix. Similar features will cluster together visually. We show this as follows:

In [ ]:
sns.clustermap(corr_matrix, 
               method='ward', 
               cmap='coolwarm', 
               annot=True)
plt.title("Clustered Heatmap of Feature Correlations")
plt.show()


# Task 3: Logistic Regression Implementation

## 1. Logistic Regression Implementation From Scratch


Because wine quality is originally a multi‐class integer (3–8), we’ll convert it into a binary classification problem  (e.g., label “1” if quality 
≥ 7, else “0”).

In [ ]:

data['label'] = (data['quality'] >= 7).astype(int)
X = data.drop(columns=['quality', 'label'])  # 11 chemical features
y = data['label'].values  
X = X.values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)


############################################################################### Logistic Regression class  ###############################################################################

class LogisticRegressionScratch:
    def __init__(self, learning_rate=0.01, max_iter=1000, tol=1e-4):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.tol = tol
        self.beta = None  # will be initialized when we fit the data
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def predict_proba(self, X):
        return self.sigmoid(X.dot(self.beta))
    
    def cost_function(self, X, y):
        N = len(y)
        y_pred = self.predict_proba(X)
        eps = 1e-10
        cost = - (1/N) * np.sum(
            y * np.log(y_pred + eps) + (1 - y) * np.log(1 - y_pred + eps)
        )
        return cost
    
    def gradient(self, X, y):
        N = len(y)
        y_pred = self.predict_proba(X)
        grad = (1/N) * X.T.dot(y_pred - y)
        return grad
    
    # Batch Gradient Descent 
    def fit_batch(self, X, y):
        N, d = X.shape
        self.beta = np.zeros(d)
        
        for i in range(self.max_iter):
            grad = self.gradient(X, y)
            beta_old = self.beta.copy()
            
            self.beta -= self.learning_rate * grad
            
            # Convergence check
            if np.linalg.norm(self.beta - beta_old) < self.tol:
                break
    
    # Stochastic Gradient Descent 
    def fit_sgd(self, X, y):
        N, d = X.shape
        self.beta = np.zeros(d)
        
        for epoch in range(self.max_iter):
            indices = np.arange(N)
            np.random.shuffle(indices)
            
            for i in indices:
                xi = X[i, :]
                yi = y[i]
                # single-sample prediction & gradient
                y_pred_i = self.sigmoid(xi.dot(self.beta))
                grad_i = (y_pred_i - yi) * xi
                
                self.beta -= self.learning_rate * grad_i
    
    # Mini-Batch Gradient Descent 
    def fit_mini_batch(self, X, y, batch_size=32):
        N, d = X.shape
        self.beta = np.zeros(d)
        
        for epoch in range(self.max_iter):
            indices = np.arange(N)
            np.random.shuffle(indices)
            
            for start in range(0, N, batch_size):
                end = start + batch_size
                batch_idx = indices[start:end]
                X_batch = X[batch_idx, :]
                y_batch = y[batch_idx]
                
                grad_batch = self.gradient(X_batch, y_batch)
                self.beta -= self.learning_rate * grad_batch
    
    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X) >= threshold).astype(int)



## 2. Gradient Descent Variants


Below, we discuss the convergence characteristics of three common forms of gradient descent in machine learning: **Batch Gradient Descent**, **Stochastic Gradient Descent**, and **Mini-Batch Gradient Descent**. These methods differ mainly in how much data they use to compute a gradient update at each iteration, influencing both computational efficiency and convergence behavior.

---

## Batch Gradient Descent (BGD)

**Updates:** Batch Gradient Descent uses the *entire dataset* (containing \(N\) samples) to compute the gradient in each update step. Consequently, the gradient calculation reflects the *true* direction of steepest descent for the entire training set.

**Convergence:**  
In practice, BGD often exhibits **smooth** and **stable** convergence because it uses the exact gradient from all data at once. However, a single update can be slow if \(N\) is large, since computing the gradient is \(O(Nd)\). Although fewer iterations may be needed overall, each iteration is computationally expensive. This method is usually more feasible for smaller datasets (Goodfellow et al., 2016).

---

## Stochastic Gradient Descent (SGD)

**Updates:** Stochastic Gradient Descent computes the gradient and updates parameters on a **single sample** \(\mathbf{x}_i\) (and label \(y_i\)) at a time. Thus, each step is \(O(d)\), as opposed to \(O(Nd)\).

**Convergence:**  
SGD can make **very fast** progress per update because it updates on a per-sample basis. However, the gradient estimate is **noisy**, causing the loss function to oscillate around a minimum rather than converging smoothly (Bottou, 2010). This method is excellent for very large datasets where computing a full batch gradient is prohibitive, although careful learning-rate scheduling is typically required (Robbins & Monro, 1951).

---

## Mini-Batch Gradient Descent (MBGD)

**Updates:** Mini-Batch Gradient Descent splits the training set into small batches (e.g., sizes 16, 32, 64, etc.). Each gradient update is computed from one mini-batch at a time, balancing the extremes of full-batch and purely stochastic methods.

**Convergence:**  
Compared to pure SGD, mini-batch updates are **less noisy** because each batch contains multiple samples. However, they still tend to converge **faster** than full-batch methods, because each update is \(O(\text{batch\_size} \times d)\) rather than \(O(Nd)\) (Goodfellow et al., 2016). This makes MBGD the **most practical** approach in many large-scale settings, particularly in deep learning (Ruder, 2016).

---

## References

- **Bottou, L.** (2010). “Large-Scale Machine Learning with Stochastic Gradient Descent.” *Proceedings of COMPSTAT'2010*, 177–186.  
  [Link](https://link.springer.com/chapter/10.1007/978-3-7908-2604-3_16)

- **Goodfellow, I., Bengio, Y., & Courville, A.** (2016). *Deep Learning.* MIT Press.  
  [Link](https://www.deeplearningbook.org)

- **Rober S.** (2016). “An overview of gradient descent optimization algorithms.” *arXiv preprint* arXiv:1609.04747.  
  [Link](https://arxiv.org/abs/1609.04747)

- **Robbins, H. & Monro, S.** (1951). “A Stochastic Approximation Method.” *The Annals of Mathematical Statistics*, 22(3): 400–407.  
  [Link](https://projecteuclid.org/euclid.aoms/1177729586)


In [ ]:
#############################################################################Train & Evaluate the Model ###############################################################################

# Create an instance
model = LogisticRegressionScratch(learning_rate=0.005, max_iter=1000, tol=1e-6)

# Batch Gradient Descent
model.fit_batch(X_train, y_train)
train_cost = model.cost_function(X_train, y_train)
test_cost  = model.cost_function(X_test, y_test)
y_pred     = model.predict(X_test)

acc = np.mean(y_pred == y_test)
print("=== Batch Gradient Descent ===")
print(f"Train Cost: {train_cost:.4f}")
print(f"Test  Cost: {test_cost:.4f}")
print(f"Test  Accuracy: {acc:.4f}\n")

# Stochastic Gradient Descent
model.fit_sgd(X_train, y_train)
train_cost_sgd = model.cost_function(X_train, y_train)
test_cost_sgd  = model.cost_function(X_test, y_test)
y_pred_sgd     = model.predict(X_test)
acc_sgd        = np.mean(y_pred_sgd == y_test)

print("=== Stochastic Gradient Descent ===")
print(f"Train Cost: {train_cost_sgd:.4f}")
print(f"Test  Cost: {test_cost_sgd:.4f}")
print(f"Test  Accuracy: {acc_sgd:.4f}\n")

# Mini-Batch Gradient Descent
model.fit_mini_batch(X_train, y_train, batch_size=32)
train_cost_mb = model.cost_function(X_train, y_train)
test_cost_mb  = model.cost_function(X_test, y_test)
y_pred_mb     = model.predict(X_test)
acc_mb        = np.mean(y_pred_mb == y_test)

print("=== Mini-Batch Gradient Descent ===")
print(f"Train Cost: {train_cost_mb:.4f}")
print(f"Test  Cost: {test_cost_mb:.4f}")
print(f"Test  Accuracy: {acc_mb:.4f}")

# Task 4: Optimization Techniques and Advanced Comparison

## 1. Optimization Algorithms

In [87]:


class LogisticRegressionScratchOpt:
    def __init__(self, 
                 lr=0.01,        # Learning rate
                 max_iter=1000, 
                 tol=1e-4, 
                 optimizer='sgd',
                 # Momentum parameters
                 beta=0.9,
                 # RMSProp parameters
                 rho=0.9,
                 eps=1e-8,
                 # Adam parameters
                 beta1=0.9,
                 beta2=0.999):
        """
        lr: base learning rate
        max_iter: max epochs/iterations
        tol: tolerance for convergence check
        optimizer: 'sgd', 'momentum', 'rmsprop', or 'adam'
        beta: momentum coefficient
        rho: RMSProp decay rate
        eps: small constant to avoid zero division
        beta1, beta2: Adam coefficients
        """
        self.lr = lr
        self.max_iter = max_iter
        self.tol = tol
        self.optimizer = optimizer
        
        # Coefficients for various methods
        self.beta = beta
        self.rho = rho
        self.eps = eps
        self.beta1 = beta1
        self.beta2 = beta2
        
        # Will be set later
        self.beta_vec = None  # parameter vector
        self.m = None         # for momentum / Adam (first moment)
        self.v = None         # for RMSProp / Adam (second moment)
        self.t = 0            # iteration counter (for Adam bias correction)
        
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def predict_proba(self, X):
        return self.sigmoid(X.dot(self.beta_vec))
    
    def cost_function(self, X, y):
        N = len(y)
        y_pred = self.predict_proba(X)
        # Add small epsilon in log to avoid numerical instability
        eps = 1e-10
        cost = - (1/N) * np.sum(
            y * np.log(y_pred + eps) + (1 - y)*np.log(1 - y_pred + eps)
        )
        return cost
    
    def gradient(self, X, y):
        """Compute gradient of the cross-entropy loss wrt. parameters."""
        N = len(y)
        y_pred = self.predict_proba(X)
        grad = (1/N) * X.T.dot(y_pred - y)  # shape (d,)
        return grad
    
    def fit(self, X, y):
        """
        Train via selected optimizer. We'll do a form of (mini-)batch or
        full-batch style here. For demonstration, let's do full-batch each epoch.
        """
        N, d = X.shape
        self.beta_vec = np.zeros(d)
        
        # For advanced optimizers, initialize moment vectors:
        self.m = np.zeros(d)  # for momentum/Adam
        self.v = np.zeros(d)  # for RMSProp/Adam
        
        prev_beta = self.beta_vec.copy()
        
        for it in range(self.max_iter):
            grad = self.gradient(X, y)
            
            # Decide update rule
            if self.optimizer == 'sgd':
                # Vanilla SGD update
                update = self.lr * grad
                
            elif self.optimizer == 'momentum':
                # v <- beta * v + lr * grad
                self.m = self.beta * self.m + (self.lr * grad)
                update = self.m
                
            elif self.optimizer == 'rmsprop':
                # v <- rho * v + (1 - rho) * grad^2
                self.v = self.rho * self.v + (1 - self.rho) * (grad**2)
                # update = lr * grad / sqrt(v + eps)
                update = self.lr * grad / (np.sqrt(self.v) + self.eps)
                
            elif self.optimizer == 'adam':
                # t: iteration count
                self.t += 1
                # m <- beta1 * m + (1 - beta1)*grad
                # v <- beta2 * v + (1 - beta2)*grad^2
                self.m = self.beta1 * self.m + (1 - self.beta1) * grad
                self.v = self.beta2 * self.v + (1 - self.beta2) * (grad**2)
                
                # Bias corrections
                m_hat = self.m / (1 - self.beta1**self.t)
                v_hat = self.v / (1 - self.beta2**self.t)
                
                update = self.lr * m_hat / (np.sqrt(v_hat) + self.eps)
            
            else:
                raise ValueError("Unknown optimizer: {}".format(self.optimizer))
            
            # Perform parameter update
            self.beta_vec -= update
            
            # Check for convergence
            if np.linalg.norm(self.beta_vec - prev_beta) < self.tol:
                break
            prev_beta = self.beta_vec.copy()
    
    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X) >= threshold).astype(int)


## 2. Multiple Evaluation Metrics



In [41]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    return acc, prec, rec, f1


In [ ]:
# Convert 'quality' into a binary label: 1 if >= 7, else 0
data['label'] = (data['quality'] >= 7).astype(int)

# 2) Separate features/labels
X = data.drop(columns=['quality', 'label']).values
y = data['label'].values

# 3) Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Optional: scale features for better optimization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

# 4) Train and compare different optimizers
optimizers = ['sgd', 'momentum', 'rmsprop', 'adam']
results = {}

for opt in optimizers:
    model = LogisticRegressionScratchOpt(
        lr=0.01,       # base learning rate
        max_iter=500, 
        tol=1e-6,
        optimizer=opt,
        beta=0.9,      # Momentum
        rho=0.9,       # RMSProp
        eps=1e-8,      # small numerical constant
        beta1=0.9,     # Adam
        beta2=0.999
    )
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test, threshold=0.5)
    
    acc, prec, rec, f1 = evaluate_metrics(y_test, y_pred)
    cost = model.cost_function(X_test, y_test)
    
    results[opt] = (acc, prec, rec, f1, cost)

# 5) Print results
for opt in optimizers:
    acc, prec, rec, f1, cost = results[opt]
    print(f"=== {opt.upper()} ===")
    print(f"Accuracy : {acc:.3f}")
    print(f"Precision: {prec:.3f}")
    print(f"Recall   : {rec:.3f}")
    print(f"F1 Score : {f1:.3f}")
    print(f"Final Cost: {cost:.4f}\n")


Interpreting Results
Accuracy: Overall fraction of correct predictions.
Precision: Among the predicted positives, how many are truly positive.
Recall: Among the truly positive samples, how many were detected as positive.
F1: Harmonic mean of precision and recall.
Cost: The final cross-entropy loss; lower is generally better.
Different optimizers might converge to slightly different parameter vectors (local minima or plateaus in the cost surface) and thus produce varying metrics. Some might train faster or more stably.


**All four optimizers**—SGD, Momentum, RMSProp, and Adam—are converging to roughly the same solution in this Wine Quality classification. They end up with very similar accuracy (0.66–0.68), a relatively low precision (~0.30), high recall (~0.98), and a final cost around 0.63–0.64.  

---

### 1. High Recall, Low Precision

The  model seems to be  classifying most wines as positive (quality \(\ge 7\)), which correctly captures nearly 98% of actual positives, hence the high recall. At the same time, it incorrectly labels a large share of negatives as positives, dragging precision down to about 0.30. This indicates a high rate of false positives.

There are a couple of reasons this might happen. First, if the dataset is **imbalanced**—say, fewer high‐quality wines than low‐quality wines—a model that predominantly guesses “positive” can achieve a high recall. Second, the default decision threshold of 0.5 might not be optimal. If the logistic regression outputs are typically above 0.5, the model will generate a large number of positives.  

---

### 2. Similar Final Results Across Optimizers

All optimizers converge to a **similar parameter solution** for logistic regression, and their final costs differ only in minor decimals (e.g., 0.6384 vs. 0.634x). This is **not unusual** for a relatively small dataset where each method finds a similar local optimum. Momentum, RMSProp, and Adam often show bigger advantages on more complex problems (like deep neural networks) or very large datasets with noisy gradients.  

---

### 3. Potential Improvements

1.  
   By default, a sample is labeled positive if \(\hat{p} \ge 0.5\). We might raise this threshold to reduce false positives and thus improve precision. Tools like the precision–recall curve or ROC curve can guide you to a better threshold.  

2. 
   Check how many wines have a “label” of 1 vs. 0


# 3. Hyperparameter Tuning Process

In [94]:
from itertools import product
import numpy as np

# We'll reuse our LogisticRegressionScratchOpt class (from Task 4).
# Also assume evaluate_metrics(...) is a function returning (acc, prec, rec, f1).

############################################################################### 1. Momentum Tuning##############################################################################
momentum_vals = [0.8, 0.9, 0.95]
learning_rates = [0.001, 0.01, 0.05]

best_score_momentum = 0
best_params_momentum = None

for m, lr in product(momentum_vals, learning_rates):
    model = LogisticRegressionScratchOpt(
        lr=lr, max_iter=500, tol=1e-6,
        optimizer='momentum',
        beta=m    # momentum coefficient
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    acc, prec, rec, f1 = evaluate_metrics(y_test, y_pred)
    if f1 > best_score_momentum:
        best_score_momentum = f1
        best_params_momentum = (m, lr)

print(f"=== Best Momentum Params ===")
print(f"F1: {best_score_momentum:.3f} with beta={best_params_momentum[0]}, lr={best_params_momentum[1]}")

############################################################################### 2. RMSProp Tuning ##############################################################################
rmsprop_rhos = [0.8, 0.9, 0.95]
learning_rates = [0.001, 0.01, 0.05]

best_score_rms = 0
best_params_rms = None

for rho, lr in product(rmsprop_rhos, learning_rates):
    model = LogisticRegressionScratchOpt(
        lr=lr, max_iter=500, tol=1e-6,
        optimizer='rmsprop',
        rho=rho  # RMSProp decay rate
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    acc, prec, rec, f1 = evaluate_metrics(y_test, y_pred)
    if f1 > best_score_rms:
        best_score_rms = f1
        best_params_rms = (rho, lr)

print(f"=== Best RMSProp Params ===")
print(f"F1: {best_score_rms:.3f} with rho={best_params_rms[0]}, lr={best_params_rms[1]}")

############################################################################### 3. Adam Tuning  ##############################################################################
adam_beta1_vals = [0.8, 0.9]
adam_beta2_vals = [0.99, 0.999]
learning_rates = [0.001, 0.01]

best_score_adam = 0
best_params_adam = None

for b1, b2, lr in product(adam_beta1_vals, adam_beta2_vals, learning_rates):
    model = LogisticRegressionScratchOpt(
        lr=lr, max_iter=500, tol=1e-6,
        optimizer='adam',
        beta1=b1,   # Adam's first-moment decay
        beta2=b2    # Adam's second-moment decay
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    acc, prec, rec, f1 = evaluate_metrics(y_test, y_pred)
    if f1 > best_score_adam:
        best_score_adam = f1
        best_params_adam = (b1, b2, lr)

print(f"=== Best Adam Params ===")
print(f"F1: {best_score_adam:.3f} with beta1={best_params_adam[0]}, beta2={best_params_adam[1]}, lr={best_params_adam[2]}")


=== Best Momentum Params ===
F1: 0.472 with beta=0.8, lr=0.001
=== Best RMSProp Params ===
F1: 0.474 with rho=0.8, lr=0.01
=== Best Adam Params ===
F1: 0.472 with beta1=0.8, beta2=0.999, lr=0.001


In [95]:
# Retrain final models using the best hyperparams found:

#Momentum
model_momentum = LogisticRegressionScratchOpt(
    lr=best_params_momentum[1],
    max_iter=500, tol=1e-6,
    optimizer='momentum',
    beta=best_params_momentum[0]
)
model_momentum.fit(X_train, y_train)
y_pred_momentum = model_momentum.predict(X_test)
acc_m, prec_m, rec_m, f1_m = evaluate_metrics(y_test, y_pred_momentum)

#RMSProp
model_rms = LogisticRegressionScratchOpt(
    lr=best_params_rms[1],
    max_iter=500, tol=1e-6,
    optimizer='rmsprop',
    rho=best_params_rms[0]
)
model_rms.fit(X_train, y_train)
y_pred_rms = model_rms.predict(X_test)
acc_r, prec_r, rec_r, f1_r = evaluate_metrics(y_test, y_pred_rms)

#Adam
model_adam = LogisticRegressionScratchOpt(
    lr=best_params_adam[2],
    max_iter=500, tol=1e-6,
    optimizer='adam',
    beta1=best_params_adam[0],
    beta2=best_params_adam[1]
)
model_adam.fit(X_train, y_train)
y_pred_adam = model_adam.predict(X_test)
acc_a, prec_a, rec_a, f1_a = evaluate_metrics(y_test, y_pred_adam)

#Print and compare final performances
print("=== Final Comparison After Hyperparam Tuning ===")
print(f"Momentum => F1: {f1_m:.3f}, Acc: {acc_m:.3f}, Prec: {prec_m:.3f}, Rec: {rec_m:.3f}")
print(f"RMSProp  => F1: {f1_r:.3f}, Acc: {acc_r:.3f}, Prec: {prec_r:.3f}, Rec: {rec_r:.3f}")
print(f"Adam     => F1: {f1_a:.3f}, Acc: {acc_a:.3f}, Prec: {prec_a:.3f}, Rec: {rec_a:.3f}")


=== Final Comparison After Hyperparam Tuning ===
Momentum => F1: 0.472, Acc: 0.678, Prec: 0.311, Rec: 0.979
RMSProp  => F1: 0.474, Acc: 0.681, Prec: 0.313, Rec: 0.979
Adam     => F1: 0.472, Acc: 0.678, Prec: 0.311, Rec: 0.979


Below is a concise **comparison** of our **final, tuned** results versus the **original, untuned** results. Overall, the changes are fairly modest, indicating that all optimizers were already converging to a similar region in parameter space for this particular logistic regression task.

---

### Original Results

| Optimizer  | Accuracy | Precision | Recall | F1 Score | Final Cost |
|------------|---------:|----------:|-------:|---------:|-----------:|
| **SGD**      | 0.678  | 0.311     | 0.979  | 0.472    | 0.6384     |
| **Momentum** | 0.666  | 0.303     | 0.979  | 0.462    | 0.6348     |
| **RMSProp**  | 0.672  | 0.307     | 0.979  | 0.467    | 0.6337     |
| **Adam**     | 0.669  | 0.305     | 0.979  | 0.465    | 0.6347     |



---

### After Hyperparameter Tuning

| Optimizer  | Accuracy | Precision | Recall | F1 Score |
|------------|---------:|----------:|-------:|---------:|
| **Momentum** | 0.678  | 0.311     | 0.979  | 0.472    |
| **RMSProp**  | 0.681  | 0.313     | 0.979  | 0.474    |
| **Adam**     | 0.678  | 0.311     | 0.979  | 0.472    |



---

### Key Observations

**First,** RMSProp appears to benefit the most from tuning, with Accuracy rising from 0.672 to 0.681 and F1 from 0.467 to 0.474. While the changes seem modest, this is a consistent improvement across metrics.

**Second,** Momentum and Adam also see small boosts. Momentum’s F1 rises from 0.462 to 0.472, and Adam’s from 0.465 to 0.472, alongside slight increases in Accuracy. Though not drastic, these  suggest the hyperparameters did matter.

**Third,** all methods still converge on a **similar pattern**: **high recall (0.979)** and relatively **low precision (~0.31)**. This indicates the model heavily predicts “positive,” leading to few false negatives but many false positives. 

**Finally,** these relatively small gains suggest that for a smaller dataset and a linear model like logistic regression, advanced optimizers plus tuning won’t massively alter final performance. Typically, **larger gains** appear in more **complex or large‐scale** tasks, where momentum‐based or adaptive learning rate methods really shine.


# 4. Practical Trade-Offs

### Practical Trade‐Offs of the Algorithms

Below is a concise overview of the key trade‐offs for **SGD**, **Momentum**, **RMSProp**, and **Adam**, focusing on **computational complexity**, **interpretability**, and **suitability for large‐scale tasks**.

1. **Vanilla SGD (Stochastic Gradient Descent)**  
   - **Computational Complexity**:  
     Per‐iteration cost is \(O(d)\) if you update after each sample (where \(d\) is the number of parameters). This is highly efficient on large datasets, because you do not need to load all data at once.  
   - **Interpretability**:  
     The model (e.g., logistic regression) remains just as interpretable, since SGD does not affect the linear form or parameter interpretability. The optimizer only changes how parameters are fit.  
   - **Suitability for Large‐Scale**:  
     Very suitable, as it can handle streaming data (one sample at a time). However, noisy updates typically need careful **learning‐rate scheduling** or other variants (Bottou, 2010).

2. **Momentum**  
   - **Computational Complexity**:  
     Nearly the same as vanilla SGD per step (still \(O(d)\)), but you must maintain and update a “velocity” vector. The overhead is minimal (just a vector addition and scalar multiplication).  
   - **Interpretability**:  
     Like SGD, momentum does not change the logistic regression’s parameter interpretation. Momentum simply uses past gradients to smooth updates, which does not affect how final coefficients are read.  
   - **Suitability for Large‐Scale**:  
     Works well in large‐scale settings, often converging faster (in fewer epochs) than vanilla SGD because it dampens oscillations and carries “inertia” in the gradient direction (Sutskever et al., 2013).

3. **RMSProp**  
   - **Computational Complexity**:  
     Maintains a running average of squared gradients (a second‐moment estimate), which adds modest overhead but remains \(O(d)\).  
   - **Interpretability**:  
     Similarly does not alter the linear model’s interpretability; it only affects optimization dynamics by adjusting the step size for each parameter proportionally to its recent gradient magnitudes.  
   - **Suitability for Large‐Scale**:  
     Highly suitable for large or noisy problems, thanks to adaptive learning rates that help with faster convergence (Tieleman & Hinton, 2012). However, hyperparameter tuning (decay rate, learning rate) can be tricky.

4. **Adam (Adaptive Moment Estimation)**  
   - **Computational Complexity**:  
     Slightly more overhead than RMSProp or Momentum, since it maintains both first‐moment (velocity) and second‐moment (squared gradients) estimates. Still \(O(d)\) per iteration, so quite feasible.  
   - **Interpretability**:  
     Like the others, Adam does not change the fundamental meaning of model coefficients. It merely changes how the parameters are updated during training.  
   - **Suitability for Large‐Scale**:  
     One of the most popular optimizers in deep learning, especially for large and complex models. It often converges quickly out of the gate and handles noisy, sparse gradients well (Kingma & Ba, 2015). Tuning \(\beta_1, \beta_2\), and the learning rate can still be necessary.

---

### Conclusion

- **Computational Complexity**: All four methods have effectively \(O(d)\) update steps for logistic regression, with minimal overhead for momentum‐ or adaptive‐based variants.  
- **Interpretability**: None of the optimizers diminish or alter the intrinsic interpretability of a linear model; they simply converge differently. For a standard logistic regression, the coefficients remain interpretable.  
- **Suitability for Large‐Scale Datasets**:  
  - **SGD** (and mini‐batch SGD) remain a robust default for very large datasets, especially if streaming or online learning is needed.  
  - **Momentum** often reduces training epochs by smoothing out SGD’s updates.  
  - **RMSProp** and **Adam** excel in noisy or high‐dimensional domains where adaptive learning rates significantly improve convergence speed. Both are widely adopted in deep learning, though they can require extra hyperparameter tuning.

In smaller or moderate‐sized problems (like the Wine Quality dataset), all optimizers can end up at very similar solutions. Differences become more pronounced as model complexity and dataset size grow.

---

### References

- **Bottou, L.** (2010). “Large-Scale Machine Learning with Stochastic Gradient Descent.” In *Proceedings of COMPSTAT'2010* (pp. 177–186).  
  [Link](https://link.springer.com/chapter/10.1007/978-3-7908-2604-3_16)

- **Kingma, D. P., & Ba, J.** (2015). “Adam: A Method for Stochastic Optimization.” In *3rd International Conference on Learning Representations (ICLR)*.  
  [Link](https://arxiv.org/abs/1412.6980)

- **Sutskever, I., Martens, J., Dahl, G., & Hinton, G.** (2013). “On the importance of initialization and momentum in deep learning.” *Proceedings of the 30th International Conference on Machine Learning (ICML)* (pp. 1139–1147).  
  [Link](http://proceedings.mlr.press/v28/sutskever13.html)

- **Tieleman, T., & Hinton, G.** (2012). “Lecture 6.5—RMSProp: Divide the gradient by a running average of its recent magnitude.” *COURSERA: Neural Networks for Machine Learning*.  
  [Link](http://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf)
